In [5]:
import pandas as pd
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv("Customer-Churn-Records.csv")

df = df.drop(columns=['RowNumber','CustomerId'])
df = df.drop(columns=['Complain'])
df = df.rename(columns={'Card Type': 'CardType'})
df = df.rename(columns={'Point Earned': 'PointsEarned'})
df = df.rename(columns={'Geography': 'Country'})
df = df.rename(columns={'Satisfaction Score': 'SatisfactionScore'})
myColumns = df.columns

<font color='lightblue'> 
We split the data now so that we dont draw conclusions from the whole dataset when transforming features. Since we want to simulate the real world occurrence of not having access to test data distribution.

We also split the data into a validation and test set so that we later can fine tune on the validation set and not risk 'data leakage' (i think that is the right term).
</font>

In [92]:
#
#* Split the dataset so that we fit_transform the training data and transform the test data

# since the target feature is a bit uneven we stratify so that it does not effect us to much at this stage
X_train, X_temp, y_train, y_temp = train_test_split(df.drop('Exited', axis=1), df['Exited'], test_size=0.2, stratify=df['Exited'], random_state=42)

# We use train_test_split twice so we can get a validation set as well
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)
# It is worth considering to use train_test_split again to get a holdout set.


print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(8000, 14)
(8000,)
(1000, 14)
(1000,)
(1000, 14)
(1000,)


In [93]:
X_train.head()

,Surname,CreditScore,Country,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,SatisfactionScore,CardType,PointsEarned
0,Taylor,709,Spain,Male,35,2,0.00,2,1,0,104982.39,2,GOLD,422
1,Kornilova,744,France,Male,29,1,43504.42,1,1,1,119327.75,1,PLATINUM,607
2,Kodilinyechukwu,773,France,Male,64,2,145578.28,1,0,1,186172.85,1,SILVER,630
3,Wang,646,Germany,Female,29,4,105957.44,1,1,0,15470.91,1,PLATINUM,345
4,Baresi,675,France,Female,57,8,0.00,2,0,1,95463.29,3,SILVER,632


<font color='0FF00'> 
Let us feature transform each feature now! 
There are several different encodings and transformations we could try.
Some of them, such as target encoding, are more experimental in nature.
</font>

<font color='skyBlue'>"Banks usually categorize credit scores into several risk profiles or categories. According to CNBC 1, credit scores are typically divided into five categories:

Deep subprime: Credit scores below 580
Subprime: Credit scores between 580 and 619
Near-prime: Credit scores between 620 and 659
Prime: Credit scores between 660 and 719
Super-prime: Credit scores of 720 or above

These categories help banks assess the creditworthiness of borrowers and determine the types of financial products and interest rates they are eligible for"</font>

In [94]:
#
#! CreditScore

# Create a dictionary mapping credit score ranges to ordinal values
encoding_dict = {
    (0, 580): 0,
    (581, 619): 1,
    (620, 659): 2,
    (660, 719): 3,
    (720, 10000): 4
}

# Define a function to apply the ordinal encoding
def apply_encoding(credit_score):
    for score_range, encoded_value in encoding_dict.items():
        if score_range[0] <= credit_score <= score_range[1]:
            return encoded_value

# Apply ordinal encoding using apply() function
X_train['CreditScoreOrd'] = X_train['CreditScore'].apply(apply_encoding)
X_test['CreditScoreOrd'] = X_test['CreditScore'].apply(apply_encoding)
X_val['CreditScoreOrd'] = X_val['CreditScore'].apply(apply_encoding)

In [96]:
#
# ! Age

#* Only difference is that the countries have different retirement ages.
#* I also considered binning the age where students start university.
age_groups = {
    'France': [(18, 24), (25, 35), (36, 45), (46, 62), (63 , 100)],
    'Germany': [(18, 24), (25, 35), (36, 45), (46, 65), (66 , 100)], 
    'Spain': [(18, 24), (25, 35), (36, 45), (46, 66), (67 , 100)],
}

#? https://www.thelocal.de/20230127/how-does-germanys-retirement-age-compare-to-the-rest-of-europe
#? https://www.cleiss.fr/docs/regimes/regime_france/an_3.html
#? https://www.caserexpatinsurance.com/blog-typical-non-spanish/retirement-age-in-spain

def get_age_groups(country, age):
    tax_ranges = age_groups[country]
    for index, tax_range in enumerate(tax_ranges):
        if tax_range[0] <= age <= tax_range[1]:
            return index
    return None

X_train['AgeOrd'] = X_train.apply(lambda row: get_age_groups(row['Country'], row['Age']), axis=1)
X_test['AgeOrd'] = X_test.apply(lambda row: get_age_groups(row['Country'], row['Age']), axis=1)
X_val['AgeOrd'] = X_val.apply(lambda row: get_age_groups(row['Country'], row['Age']), axis=1)

<font color='skyBlue'> 
I discretize EstimatedSalary salary based on tax brackets that are used in the respective country of the member.

Spain has one more bracket compared to France and Germany, so there are multiple ways to handle this. I just merged the top brackets.
</font>

In [97]:
#
#! EstimatedSalary

tax_brackets = {
    'France': [(0, 10_225), (10_226, 26_070), (26_071, 74_545), (74_546, 160_336), (16_0337, float('inf'))], # For 2022
    'Germany': [(0, 10_347), (10_348, 15_999), (16000, 62809), (62810, 277825), (277826, float('inf'))], # For 2023. Also the source has a gap between the first brackets that i filled in (Up to 10,347. 10,909-15,999)
    'Spain': [(0, 12_450), (12451, 20200), (20201, 35200), (35201, 100000), (100001, float('inf'))], # For 2021. Spain has 6 brackets, so we are combining the highest 2
}

def get_tax_bracket(country, salary):
    tax_ranges = tax_brackets[country]
    for index, tax_range in enumerate(tax_ranges):
        if tax_range[0] <= salary <= tax_range[1]:
            return index
    return None

X_train['TaxBracket'] = X_train.apply(lambda row: get_tax_bracket(row['Country'], row['EstimatedSalary']), axis=1)
X_test['TaxBracket'] = X_test.apply(lambda row: get_tax_bracket(row['Country'], row['EstimatedSalary']), axis=1)
X_val['TaxBracket'] = X_val.apply(lambda row: get_tax_bracket(row['Country'], row['EstimatedSalary']), axis=1)


#? https://www.worldwide-tax.com/france/france-taxes.asp
#? https://www.worldwide-tax.com/spain/spain-taxes.asp
#? https://www.worldwide-tax.com/germany/germany-taxes.asp

<font color='skyBlue'> 
The important thing with Balance is that zero balance gets its own category, since its probable that it correlates uniquely with other features. Does not have to mean that the customer has little money, perhaps they are not using the bank to save any money at all
</font>

In [98]:
#
#! Balance
#* I grouped according to the distribution (so that the split between 50% is between group 3 and 4)
encoding_dict = {
    (-1, 1): 0,
    (2, 100_000): 1,
    (100_001, 125_000): 2,
    (125_001, 149_999): 3,
    (150_000, 300_000): 4
}

# Define a function to apply the ordinal encoding
def apply_encoding(balance):
    for balance_range, encoded_value in encoding_dict.items():
        if balance_range[0] <= balance <= balance_range[1]:
            return encoded_value

# Apply ordinal encoding using apply() function
X_train['BalanceOrd'] = X_train['Balance'].apply(apply_encoding)
X_test['BalanceOrd'] = X_test['Balance'].apply(apply_encoding)
X_val['BalanceOrd'] = X_val['Balance'].apply(apply_encoding)

<font color='skyBlue'> 
Have not found anything that points earned correlates with any other feature
</font>

In [99]:
#
#! PointsEarned
#* I decided to bin PointsEarned using the quntiles

PointsEarnedValues = X_train['PointsEarned'].values.reshape(-1, 1)
PointsEarnedValues_test = X_test['PointsEarned'].values.reshape(-1, 1)
PointsEarnedValues_val = X_val['PointsEarned'].values.reshape(-1, 1)

# Create a KBinsDiscretizer object with 5 bins and the quantile strategy
discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')

# Fit and transform the data using the discretizer
#* I make sure to fit on the val and test sets and only transform
PointsEarnedEnc = discretizer.fit_transform(PointsEarnedValues)
PointsEarnedEnc_test = discretizer.transform(PointsEarnedValues_test)
PointsEarnedEnc_val = discretizer.transform(PointsEarnedValues_val)

# Assign the discretized values to a new column in the DataFrame
X_train['PointsEarnedQuant'] = PointsEarnedEnc.flatten()
X_test['PointsEarnedQuant'] = PointsEarnedEnc_test.flatten()
X_val['PointsEarnedQuant'] = PointsEarnedEnc_val.flatten()

<font color='skyBlue'> 
Let us do the same with credit card
</font>

In [100]:
#
#! CardType Credit card type

# Ordinal encoding based on cardType value level makes sense
CardTypeValues = X_train['CardType'].values.reshape(-1, 1)
CardTypeValues_test = X_test['CardType'].values.reshape(-1, 1)
CardTypeValues_val = X_val['CardType'].values.reshape(-1, 1)

# Define the order of the cards as a list
card_order = ['SILVER', 'GOLD', 'PLATINUM', 'DIAMOND']

# Create an OrdinalEncoder object with the desired order
encoder = OrdinalEncoder(categories=[card_order])

# Fit and transform the data using the encoder
#! this is not necessary since we are not fitting a distribution
CardTypeEnc = encoder.fit_transform(CardTypeValues)
CardTypeEnc_test = encoder.transform(CardTypeValues_test)
CardTypeEnc_val = encoder.transform(CardTypeValues_val)

# Assign the encoded values to a new column in the DataFrame
X_train['CardTypeOrd'] = CardTypeEnc.flatten()
X_test['CardTypeOrd'] = CardTypeEnc_test.flatten()
X_val['CardTypeOrd'] = CardTypeEnc_val.flatten()


<font color='skyBlue'> 
Even though we did not find a significant correlation between name count and Exited we will try frequency encoding, since there could be some more complex relationships that the models could uncover
</font>

In [102]:
#
#* Ordinal encoding of Surname based on name count (only the 10 most common names), less common names get encoded as 0

# Calculate the frequency of occurrence for each value in the 'Surname' column
value_frequencies = X_train['Surname'].value_counts()

# Select the top 11 most common names
top_names = value_frequencies.nlargest(11).index

# Reverse index so that most common name gets encoded as the highest value
top_names = top_names[::-1]

# Create a dictionary mapping the top names to their ordinal values (0 to 9)
encoding_dict = {name: i for i, name in enumerate(top_names)}

# Use the dictionary to map the 'Surname' column values to their ordinal values in a new column,
# assigning 0 to the remaining names

X_train['SurnameOrd'] = X_train['Surname'].map(encoding_dict).fillna(0)
X_test['SurnameOrd'] = X_test['Surname'].map(encoding_dict).fillna(0)
X_val['SurnameOrd'] = X_val['Surname'].map(encoding_dict).fillna(0)

<font color='skyBlue'> Purchasing power parities (PPPs) are the rates of currency conversion that try to equalise the purchasing power of different currencies, by eliminating the differences in price levels between countries. The basket of goods and services priced is a sample of all those that are part of final expenditures: final consumption of households and government, fixed capital formation, and net exports. This indicator is measured in terms of national currency per US dollar.   </font>

In [103]:
#
#* Let us try to ordinal encode country based on PPP.
countryPPP = {'Germany': 0.913, 'Spain':0.742, 'France': 0.901}
#* Let us just get the order from this rather than teh PPP value
countryPPP = {'Germany': 3, 'Spain':1, 'France': 2}
#! Could be a bad idea since we are pretty sure that Germans are leaving the bank the most.

X_train['CountryOrd'] = X_train['Country'].map(countryPPP)
X_test['CountryOrd'] = X_test['Country'].map(countryPPP)
X_val['CountryOrd'] = X_val['Country'].map(countryPPP)


#? https://data.oecd.org/conversion/purchasing-power-parities-ppp.htm

<font color='skyBlue'> 
While we are at it. Le us try to add some wellbeing scores for each country since happiness is highly correalted with your economy (more on the lower end though)
</font>

In [104]:
#
#* Let us try to ordinal encode country based on PPP.
countryHappyScore = {'Germany': 7.03, 'Spain':6.48, 'France': 6.69}
#* Let us just get the order from this rather than teh PPP value
countryHappyScoreOrd = {'Germany': 3, 'Spain':1, 'France': 2}

X_train['CountryHappy'] = X_train['Country'].map(countryHappyScore)
X_test['CountryHappy'] = X_test['Country'].map(countryHappyScore)
X_val['CountryHappy'] = X_val['Country'].map(countryHappyScore)

#! Since the countries will get the same ordinal scores as they did with the PPP encoding it will not be able to uncover any none-linear patterns. So we either encode a different way (e.g. proportional or keep the happy number) or dont add this feature it all

#? https://wisevoter.com/country-rankings/happiest-countries-in-the-world/


In [105]:
#
#* Gender is the only binary feature that is in string format
X_train['GenderBinary'] = X_train['Gender'].apply(lambda x: 0 if x == 'Male' else 1)
X_test['GenderBinary'] = X_test['Gender'].apply(lambda x: 0 if x == 'Male' else 1)
X_val['GenderBinary'] = X_val['Gender'].apply(lambda x: 0 if x == 'Male' else 1)


<font color='skyBlue'> 
Tenure and NumofProducts are ordinal by default so no need to transorm (to clarify you dont need to ordinal encode since, usually one-hot is good option)

HasCrCard, IsActiveMember, Complain are already label encoded (which is also one-hot in this case). Also our target feature Exited does also not need a transformation.
</font>

<font color='skyBlue'> 
Let us end this stage by dropping the features that have been transformed
</font>

In [108]:
#
#* We are dropping every feature that has been transformed. The features that dont need transformation are kept
X_train = X_train.drop(['Surname', 'CreditScore', 'Country', 'Age', 'Balance', 'EstimatedSalary', 'CardType', 'PointsEarned', 'Gender'], axis=1)
X_test = X_test.drop(['Surname', 'CreditScore', 'Country', 'Age', 'Balance', 'EstimatedSalary', 'CardType', 'PointsEarned', 'Gender'], axis=1)
X_val = X_val.drop(['Surname', 'CreditScore', 'Country', 'Age', 'Balance', 'EstimatedSalary', 'CardType', 'PointsEarned', 'Gender'], axis=1)

# reset index
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)

#* saving the transformed dataFrame
X_train.to_csv('X_train_tran.csv', index=False)
X_test.to_csv('X_test_tran.csv', index=False)
y_train.to_csv('y_train_tran.csv', index=False)
y_test.to_csv('y_test_tran.csv', index=False)
X_val.to_csv('X_val_tran.csv', index=False)
y_val.to_csv('y_val_tran.csv', index=False)
#* The y_test and y_train has not really been transformed but i decided to group them based on name.

In [109]:
X_train.head(20)

,Tenure,NumOfProducts,HasCrCard,IsActiveMember,SatisfactionScore,CreditScoreOrd,AgeOrd,TaxBracket,BalanceOrd,PointsEarnedQuant,CardTypeOrd,SurnameOrd,CountryOrd,CountryHappy,GenderBinary
0,2,2,1,0,2,3,1,4,0,1.0,1.0,0.0,1,6.48,0
1,1,1,1,1,1,4,1,3,1,2.0,2.0,0.0,2,6.69,0
2,2,1,0,1,1,4,4,4,3,2.0,0.0,0.0,2,6.69,0
3,4,1,1,0,1,2,1,1,2,0.0,2.0,2.0,3,7.03,1
4,8,2,0,1,3,3,3,3,0,2.0,0.0,0.0,2,6.69,1
5,10,1,1,0,1,4,3,3,1,3.0,2.0,0.0,3,7.03,0
6,2,1,1,1,3,0,1,3,3,1.0,2.0,0.0,3,7.03,0
7,1,1,1,1,3,3,3,3,1,4.0,2.0,0.0,3,7.03,0
8,1,2,0,0,2,0,2,3,4,0.0,0.0,0.0,2,6.69,1
9,6,2,1,1,1,2,1,3,0,3.0,2.0,0.0,1,6.48,1


In [110]:
X_train.columns

Index(['Tenure', 'NumOfProducts', 'HasCrCard', 'IsActiveMember',
       'SatisfactionScore', 'CreditScoreOrd', 'AgeOrd', 'TaxBracket',
       'BalanceOrd', 'PointsEarnedQuant', 'CardTypeOrd', 'SurnameOrd',
       'CountryOrd', 'CountryHappy', 'GenderBinary'],
      dtype='object')